In [10]:
import json
import os
import time
import torch
import torch.utils.data
import torch.optim as optim
from torchvision import transforms
from my_dataset import MyDataSet
from utils import split_data, create_lr_scheduler, get_params_groups, train_one_epoch, evaluate
from torchinfo import summary

In [15]:
device = 'cuda:0'
data_path = "../2018.01/GOLD_XYZ_OSC.0001_1024.hdf5"
snr = 30
batch_size = 1
nw = min([os.cpu_count(), batch_size if batch_size > 1 else 0, 8])  # number of workers
num_classes = 24
model = "convnext"
if model == 'convnext':
    from model import convnext as create_model
elif model == "convnet":
    from models.convnet import ConvNet as create_model
    L=2
    C=25
elif model == "convnetBN":
    from models.convnet import ConvNetBN as create_model
    L=2
    C=25

In [16]:
device = torch.device(device if torch.cuda.is_available() else "cpu")
print(f"using {device} device...")

data_dir = os.path.dirname(os.path.abspath(data_path))
# train_indexes, train_labels, val_indexes, val_labels
reload_data = False
if os.path.exists(data_dir + f'/train_indexes{snr}.json') and not reload_data:
    with open(data_dir + f'/val_indexes{snr}.json') as f:
        val_indexes = json.load(f)
    with open(data_dir + f'/val_label{snr}.json') as f:
        val_labels = json.load(f)
else:
    train_indexes, train_labels, val_indexes, val_labels = split_data(data_path, snr,
                                                                      ratio=[0.875, 0.125, 0.],
                                                                      test=False, one_hot=False)

print("using valid data size: {}".format(len(val_labels)))
data_transform = {
    "val": transforms.Compose([transforms.ToTensor()])
}

val_dataset = MyDataSet(hdf5_path=data_path,
                        mod_class=val_labels,
                        indexes=val_indexes,
                        model=model,
                        transform=data_transform["val"])

using cuda:0 device...
using valid data size: 12288


In [17]:
val_dataset[0][0].shape

torch.Size([1, 1024, 2])

In [19]:
# create model
if "convnet" in model:
    net = create_model(num_classes=num_classes, L=L, C=C).to(device)
    # load model weights
    model_weight_path = f"./weights/{model}_{L}_{C}/snr{snr}.pth"
    print("Loading from ", model_weight_path)
else:
    net = create_model(num_classes=num_classes).to(device)
    # load model weights
    model_weight_path = f"./weights/{model}/snr{snr}.pth"
net.eval()

summary(net, input_size=(batch_size, 1, 1024, 2), device='cpu')

Layer (type:depth-idx)                   Output Shape              Param #
ConvNeXt                                 [1, 24]                   --
├─Conv2d: 1-1                            [1, 32, 1024, 1]          96
├─ModuleList: 1-8                        --                        (recursive)
│    └─Sequential: 2-1                   [32, 1024]                --
│    │    └─LayerNorm: 3-1               [32, 1024]                65,536
├─ModuleList: 1-9                        --                        (recursive)
│    └─Sequential: 2-2                   [1, 32, 1024]             --
│    │    └─Block: 3-2                   [1, 32, 1024]             74,304
│    │    └─Block: 3-3                   [1, 32, 1024]             74,304
│    │    └─Block: 3-4                   [1, 32, 1024]             74,304
├─ModuleList: 1-8                        --                        (recursive)
│    └─Sequential: 2-3                   [1, 64, 256]              --
│    │    └─LayerNorm: 3-5               [

In [6]:
with torch.no_grad():
    # predict class
    output = model(data.to(device))

In [7]:
output.shape

torch.Size([1, 24])

In [8]:
output

tensor([[23.9752,  7.1303,  1.6010,  9.3841, -4.8871, -6.6282, -0.0838, -0.1094,
          2.5885, 14.3770, -1.9049,  7.3600, -2.4526,  3.9232, -7.7196, -4.7046,
         -7.8209,  9.3376,  0.0529,  1.7865,  6.8980, -2.0233, -3.0467, -6.4652]],
       device='cuda:0')

In [12]:
torch.max(output, dim=1)[1]

tensor([0], device='cuda:0')